In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../data/raw/BTCUSDT.csv")
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Save in df the columns 'timestamp', 'open', 'high', 'low', 'close', 'volume' 
df = df[['timestamp', 'close', 'volume', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume']]
df

# Only save the rows where the timestamp is the biggest value of the of the day


In [ ]:
# Convert data to DataFrame
df = pd.DataFrame(df)

# Convert the string timestamp column to datetime
df['date'] = pd.to_datetime(df['timestamp'])

df

In [ ]:
df.drop('timestamp', axis=1, inplace=True)
df

In [ ]:
# Check for the last row of the day
# Convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Extract the date and time separately
df['date_only'] = df['date'].dt.date
df['time_only'] = df['date'].dt.time

# Group by date and get the last entry for each day
last_time_of_day = df.groupby('date_only').tail(1)

last_time_of_day
last_time_of_day.to_csv('../data/raw/last_time_of_day.csv', index=False)
df = pd.read_csv('../data/raw/last_time_of_day.csv')
df


In [ ]:
# Creating a new column with the promedio of the open and close columns
#df['promedio'] = (df['open'] + df['close']) / 2
df

In [ ]:
# Set training data 
X = df[['volume', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume']]
Y = df['close']

X.shape, Y.shape


In [ ]:
# Split the data into training and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [ ]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

In [ ]:
# Linear refression model
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score


In [ ]:
# Builing model
model = linear_model.LinearRegression()
model.fit(X_train, Y_train)


In [ ]:
# Apply trained model to make prediction (on test set)
Y_pred = model.predict(X_test)

In [ ]:
print('Coefficients: \n', model.coef_)
print('Intercept: \n', model.intercept_)
print('Mean squared error: %.2f' % mean_squared_error(Y_test, Y_pred))
print('Coefficient of determination: %.2f' % r2_score(Y_test, Y_pred))

print()

In [ ]:
# Graphical representation
import matplotlib.pyplot as plt

plt.scatter(Y_test, Y_pred)
plt.xlabel('True values')
plt.ylabel('Predictions')
plt.show()



In [ ]:
# Now, creating a random forest model to predict the promedio column and future values on time
from sklearn.ensemble import RandomForestRegressor

# Building model
model = RandomForestRegressor(n_estimators=100, random_state=2)
model.fit(X_train, Y_train)

# Apply trained model to make prediction (on test set)
Y_pred = model.predict(X_test)

print('Mean squared error: %.2f' % mean_squared_error(Y_test, Y_pred))
print('Coefficient of determination: %.2f' % r2_score(Y_test, Y_pred))

# Saving the predicted data on a csv file
df = pd.DataFrame({'True': Y_test, 'Predicted': Y_pred})
df.to_csv('../data/processed/true_vs_predicted.csv', index=False)



# Print the difference between the true values and the predictions
print(Y_test - Y_pred)





In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Assuming df is your DataFrame and it has columns 'date' and 'value'
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].map(dt.datetime.toordinal)

X = df['date'].values.reshape(-1,1)
y = df['close']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)

# Predict a value for a date larger than those in the dataset
future_date = pd.to_datetime('2025-01-01').toordinal()
predicted_value = model.predict(np.array([[future_date]]))

print(predicted_value)